In [1]:
import pandas as pd

In [2]:
from fastapi import FastAPI, Query
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Cargo los archivos
items = pd.read_parquet('Dataset/items_clean.parquet')
games = pd.read_parquet('Dataset/games_clean.parquet')
reviews = pd.read_parquet('Dataset/reviews_clean.parquet')

In [4]:
async def developer(developer : str = Query(default='Valve')):
    games = pd.read_parquet('./Datasets/developer.parquet')
    # Filtramos por el desarrollador ingresado.
    games = games[games['developer'] == developer]
    # Creamos la columna año en base a la columna release_date.
    games['Año'] = games['release_date'].games.year
    # Agrupamos por año y contamos la cantidad de items.
    games_year = games.groupby('Año').size().reset_index(name='Cantidad de Items')
    # Agrupamos por año y sumamos la cantidad de items free.
    games_free = games.groupby('Año')['free'].sum().reset_index(name='free')
    # Hacemos un merge de ambos dataframes mediante la columna Año.
    games = pd.merge(games_year, games_free, on='Año')
    # Calculamos el porcentaje de contenido free.
    games['Contenido Free'] = round(games['free'] / games['Cantidad de Items'] * 100, 2)
    # Eliminamos la columna free.
    games.drop(columns=['free'], inplace=True)
    # Convertimos el dataframe a un diccionario.
    games = games.to_dict('records')
    # Devolvemos el resultado.
    return games

1) endpoint:
- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:
- Año	Cantidad de Items	Contenido Free
- 2023	50	27%
- 2022	45	25%
- xxxx	xx	xx%



2) enpoint:
- def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
- Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

3) enpoint:
- def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
- Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
def UserForGenre(genero :str) -> dict:
    genre = genre.capitalize()
    genero = games[games[genre] == 1]
    max_player_user = genero.loc[genero['playtime_forever'].idxmax(), 'user_id']
    year_playtime =genero.groupby('year')['playtime_forever'].sum().reset_index()
    playtime_list = year_playtime.to_dict(orient='records')
    result = {
        "Usuario con más horas jugadas para el Genero" + genre: max_player_user,
        "Horas jugadas": playtime_list }
    return result

genero = "Action"
resultado = UserForGenre1(genero)
print(resultado)


In [ ]:
def UserForGenre1(genero: str) -> dict:
    genre = genero.capitalize()
    filtered_games = games[games[genre] == 1]  # Filtrar por el género específico
    max_player_user = filtered_games.loc[filtered_games['playtime_forever'].idxmax(), 'user_id']  # Obtener usuario con más horas jugadas
    year_playtime = filtered_games.groupby('year')['playtime_forever'].sum().reset_index()  # Sumar horas jugadas por año
    playtime_list = year_playtime.to_dict(orient='records')  # Convertir a lista de diccionarios
    result = {
        f"Usuario con más horas jugadas para el género {genre}": max_player_user,
        "Horas jugadas por año": playtime_list
    }
    return result

# Llamada a la función para el género 'Indie'
genero = "Action"
resultado = UserForGenre1(genero)
print(resultado)


In [ ]:
genero = "Action"
resultado = UserForGenre(genero)
resultado

4) enpoint:
- def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   title         22530 non-null  object 
 3   price         22530 non-null  float32
 4   id            22530 non-null  float64
 5   developer     22530 non-null  object 
 6   release_year  22530 non-null  object 
dtypes: float32(1), float64(1), object(5)
memory usage: 1.3+ MB


In [ ]:
def best_developer_year(año :int):
    game = games[['developer', 'title', 'id', 'genres', 'release_year']]
    game = game[game['release_year'] == año]
    review = reviews[['item_id', 'positivo']]
    reviews_fielter = game.merge(review, left_on='id', right_on='item_id', how= 'inner')
    reviews_fielter = reviews_fielter[['developer', 'title','positivo']]
    reviews_fielter = reviews_fielter.groupby(['developer', 'title']).agg({'positivo':'sum'}).reset_index()
    reviews_fielter = reviews_fielter.sort_values(by='positivo', ascending= False)
    
    developer = list(reviews_fielter.iloc[:,0])
    
    return [
        {'Puesto 1': {'Developer': developer[0]}},
        {'Puesto 2': {'Developer': developer[1]}},
        {'Puesto 3': {'Developer': developer[2]}}
    ]
    #[{'Puesto 1': ['Developer:':developer[0]]}, {'Puesto 2': ['Developer:':developer[1]]}, {'Puesto 3': ['Developer:':developer[2]]}]

In [ ]:
games['id'] = games['id'].astype(str)
reviews['item_id'] = reviews['item_id'].astype(str)


In [ ]:
def best_developer_year(año: int):
    game = games[['developer', 'title', 'id', 'genres', 'release_year']]
    
    # Convertir el año proporcionado a string para la comparación
    año_str = str(año)
    
    # Filtrar juegos por release_year igual al año proporcionado
    game = game[game['release_year'] == año_str]
    
    review = reviews[['item_id', 'positivo']]  # Asumiendo que `reviews` está definido globalmente
    reviews_filter = game.merge(review, left_on='id', right_on='item_id', how='inner')
    reviews_filter = reviews_filter[['developer', 'title', 'positivo']]
    reviews_filter = reviews_filter.groupby(['developer', 'title']).agg({'positivo': 'sum'}).reset_index()
    reviews_filter = reviews_filter.sort_values(by='positivo', ascending=False)
    
    if len(reviews_filter) == 0:
        return "No hay datos disponibles para el año proporcionado."

    developers = list(reviews_filter.iloc[:, 0])

    # Verificar que hay suficientes desarrolladores para los primeros puestos
    if len(developers) < 3:
        return "No hay suficientes desarrolladores con reseñas positivas para generar los primeros puestos."

    # Retornar los primeros tres desarrolladores
    return [
        {'Puesto 1': {'Developer': developers[0]}},
        {'Puesto 2': {'Developer': developers[1]}},
        {'Puesto 3': {'Developer': developers[2]}}
    ]

# Ejemplo de uso
resultado = best_developer_year(2018)
print(resultado)


No hay datos disponibles para el año proporcionado.


In [ ]:
resultado = best_developer_year(2018)
print(resultado)

No hay datos disponibles para el año proporcionado.


5) enpoint:
- def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
- Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}